# Hi kagglers 🙋🏻‍♂️ and Welcome to this new competition!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import random
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv')
y = train['target']

# Let's do some Exploratory Data Analysis (EDA)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
#Check if there'is null values
train.isnull().sum()

In [ ]:
#Check if there'is null values
test.isnull().sum()

In [ ]:
train.describe()

In [ ]:
continous_cols= ['f'+str(i) for i in range(242)]
continous_cols.remove('f22')
continous_cols.remove('f43')
categorical_cols = ['f'+str(i) for i in range(242,285)]+['f22','f43']
cols = continous_cols + categorical_cols

In [ ]:
# plot the first 60 features 
i = 1
plt.figure()
fig, ax = plt.subplots(15, 4,figsize=(20, 22))
for feature in continous_cols[:60]:
    plt.subplot(15, 4,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
# Target distibution
sns.catplot(x="target", kind="count", palette="ch:.25", data=train)

# Modeling

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
train[continous_cols] = scaler.fit_transform(train[continous_cols])
test[continous_cols] = scaler.transform(test[continous_cols])

In [ ]:
params={'reg_alpha': 8.784125077358365, 'reg_lambda': 0.0025286925777068953, 'colsample_bytree': 0.2, 'subsample': 0.5, 'learning_rate': 0.025,
        'max_depth': 100, 'num_leaves': 7, 'min_child_samples': 185, 'cat_smooth': 54,'objective': 'binary',  
            'random_state': 48,'n_estimators': 20000,'n_jobs': -1}

In [ ]:
preds = np.zeros(test.shape[0])
kf = StratifiedKFold(n_splits=10,random_state=48,shuffle=True)
auc=[]  # list contains auc for each fold
n=0
for trn_idx, test_idx in kf.split(train[cols],y):
    X_tr,X_val=train[cols].iloc[trn_idx],train[cols].iloc[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    model = LGBMClassifier(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,eval_metric="auc",verbose=False)
    preds += model.predict_proba(test[cols])[:,1]/kf.n_splits
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))
    gc.collect()
    print(f"fold: {n+1}, auc: {auc[n]}")
    n+=1  

In [ ]:
np.mean(auc)

In [ ]:
# most 40 important features for lgb model
from optuna.integration import lightgbm as lgb
lgb.plot_importance(model, max_num_features=40, figsize=(10,10))
plt.show()

# Making a Submission

In [ ]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)

In [ ]:
sub

# I hope that you find this kernel usefull🏄